In [45]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
dbname = os.getenv('DB_NAME')

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

engine = create_engine(DATABASE_URL)

In [46]:
# Se definen todos los dataframes 
df_customers = pd.read_sql("SELECT * FROM customer",engine)
df_geolocation = pd.read_sql("SELECT * FROM geolocation",engine)
df_order_items = pd.read_sql("SELECT * FROM transformed_order_items",engine)
df_order_payments = pd.read_sql("SELECT * FROM transformed_order_payments",engine)
df_order_reviews = pd.read_sql("SELECT * FROM order_reviews",engine)
df_orders = pd.read_sql("SELECT * FROM transformed_orders",engine)
df_products = pd.read_sql("SELECT * FROM products",engine)
df_sellers = pd.read_sql("SELECT * FROM sellers",engine)
df_products_translated = pd.read_sql("SELECT * FROM products_translated", engine)

In [47]:
# Se hace merge de las tablas relevantes para el modelo de clustering con ML
df_orders = df_orders[df_orders["order_status"] == "delivered"]
df_with_merges = df_orders.merge(df_order_items, how=("left"))
df_with_merges = df_with_merges.merge(df_order_payments, how=("left"))
df_with_merges = df_with_merges.merge(df_customers, how=("left"))
df_with_merges = df_with_merges.merge(df_customers, how=("left"))

In [48]:
# Se crean columnas relevantes para el modelo
total_spent = df_with_merges.groupby("customer_unique_id")["payment_value"].sum()
avg_ticket = df_with_merges.groupby("customer_unique_id")["payment_value"].mean()
max_ticket = df_with_merges.groupby("customer_unique_id")["payment_value"].max()
total_orders = df_with_merges.groupby("customer_unique_id")["order_id"].nunique()
total_items = df_with_merges.groupby("customer_unique_id")["products_quantity"].count()

# Se crea una vista sin suplicados para sacar el average de ordenes
df_sin_duplicados = df_with_merges.drop_duplicates(subset=["customer_unique_id", "order_id"])
avg_items_per_order = df_sin_duplicados.groupby("customer_unique_id")["products_quantity"].mean()

# Primera y ultima fecha de compra de cada cliente
first_purchase_date = df_with_merges.groupby("customer_unique_id")["order_purchase_timestamp"].min()
last_purchase_date = df_with_merges.groupby("customer_unique_id")["order_purchase_timestamp"].max()

In [49]:
# Se crea otra vista ya que habia conflicto con una columna
df_order_payments_copy = df_order_payments.drop(columns="payment_value")
payment_types = df_orders.merge(df_order_payments_copy, how="left")
payment_types = payment_types.merge(df_customers, how="left")

# Se crean columnas de metodos de pago y mas usados por cada cliente
unique_payment_types = payment_types.groupby("customer_unique_id")["payment_type"].nunique()
most_used_payment_type = payment_types.groupby("customer_unique_id")["payment_type"].apply(lambda x : x.mode().iloc[0])


In [50]:
# Se migran a formato dataframe

total_spent = pd.DataFrame(total_spent).reset_index()
avg_ticket = pd.DataFrame(avg_ticket).reset_index()
max_ticket = pd.DataFrame(max_ticket).reset_index()
total_orders = pd.DataFrame(total_orders).reset_index()
total_items = pd.DataFrame(total_items).reset_index()
avg_items_per_order = pd.DataFrame(avg_items_per_order).reset_index()
unique_payment_types = pd.DataFrame(unique_payment_types).reset_index()
most_used_payment_type = pd.DataFrame(most_used_payment_type).reset_index()
first_purchase_date = pd.DataFrame(first_purchase_date).reset_index()
last_purchase_date = pd.DataFrame(last_purchase_date).reset_index()

In [51]:
# Cambian el nombre de las columnas a los mas apropiados

total_spent = total_spent.rename(columns={"payment_value": "total_spent"})
avg_ticket = avg_ticket.rename(columns={"payment_value": "avg_ticket"})
max_ticket = max_ticket.rename(columns={"payment_value": "max_ticket"})
total_orders = total_orders.rename(columns={"order_id": "total_orders"})
total_items = total_items.rename(columns={"products_quantity": "total_items"})
avg_items_per_order = avg_items_per_order.rename(columns={"products_quantity": "avg_items_per_order"})
unique_payment_types = unique_payment_types.rename(columns={"payment_type": "unique_payment_types"})
most_used_payment_type = most_used_payment_type.rename(columns={"payment_type": "most_used_payment_type"})
first_purchase_date = first_purchase_date.rename(columns={"order_purchase_timestamp": "first_purchase_date"})
last_purchase_date = last_purchase_date.rename(columns={"order_purchase_timestamp": "last_purchase_date"})

In [52]:
# Se junta la info de todos los DFs para un unico y principal respecto al modelo

df_ml = total_spent.merge(avg_ticket, on="customer_unique_id", how="left")\
    .merge(max_ticket, on="customer_unique_id", how="left")\
    .merge(total_orders, on="customer_unique_id", how="left")\
    .merge(total_items, on="customer_unique_id", how="left")\
    .merge(avg_items_per_order, on="customer_unique_id", how="left")\
    .merge(unique_payment_types, on="customer_unique_id", how="left")\
    .merge(most_used_payment_type, on="customer_unique_id", how="left")\
    .merge(first_purchase_date, on="customer_unique_id", how="left")\
    .merge(last_purchase_date, on="customer_unique_id", how="left")


In [53]:
# Se cambian a formato fechas para operar luego
df_ml['first_purchase_date'] = pd.to_datetime(df_ml['first_purchase_date'].astype(str).str[:19])
df_ml['last_purchase_date'] = pd.to_datetime(df_ml['last_purchase_date'].astype(str).str[:19])

In [54]:
# Diferencia de dias entre la primera y ultima compra
df_ml['first_last_difference_days'] = (df_ml['last_purchase_date'] - df_ml['first_purchase_date']).dt.days

In [55]:
# Columna del numero de dias de la ultima compra hasta la ultima fecha del dataset 
df_ml['recency_days'] = (df_ml["last_purchase_date"].max() -  df_ml["last_purchase_date"]).dt.days

In [56]:
# Se añaden columnas de location
df_clientes_unicos = df_customers[["customer_unique_id", "customer_state", "customer_city"]].drop_duplicates(subset="customer_unique_id")
df_ml = df_ml.merge(df_clientes_unicos[["customer_unique_id", "customer_state", "customer_city"]],how="left")

In [57]:
df_ml

,customer_unique_id,total_spent,avg_ticket,max_ticket,total_orders,total_items,avg_items_per_order,unique_payment_types,most_used_payment_type,first_purchase_date,last_purchase_date,first_last_difference_days,recency_days,customer_state,customer_city
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90,141.90,141.90,1,1,1.0,1,credit_card,2018-05-10 10:56:27,2018-05-10 10:56:27,0,111,SP,cajamar
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19,27.19,27.19,1,1,1.0,1,credit_card,2018-05-07 11:11:27,2018-05-07 11:11:27,0,114,SP,osasco
2,0000f46a3911fa3c0805444483337064,86.22,86.22,86.22,1,1,1.0,1,credit_card,2017-03-10 21:05:03,2017-03-10 21:05:03,0,536,SC,sao jose
3,0000f6ccb0745a6a4b88665a16c9f078,43.62,43.62,43.62,1,1,1.0,1,credit_card,2017-10-12 20:29:41,2017-10-12 20:29:41,0,320,PA,belem
4,0004aac84e0df4da2b147fca70cf8255,196.89,196.89,196.89,1,1,1.0,1,credit_card,2017-11-14 19:45:42,2017-11-14 19:45:42,0,287,SP,sorocaba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93352,fffcf5a5ff07b0908bd4e2dbc735a684,4134.84,2067.42,2067.42,1,2,2.0,1,credit_card,2017-06-08 21:00:36,2017-06-08 21:00:36,0,446,PE,sanharo
93353,fffea47cd6d3cc0a88bd621562a9d061,84.58,84.58,84.58,1,1,1.0,1,credit_card,2017-12-10 20:07:56,2017-12-10 20:07:56,0,261,BA,feira de santana
93354,ffff371b4d645b6ecea244b27531430a,112.46,112.46,112.46,1,1,1.0,1,credit_card,2017-02-07 15:49:16,2017-02-07 15:49:16,0,567,MT,sinop
93355,ffff5962728ec6157033ef9805bacc48,133.69,133.69,133.69,1,1,1.0,1,credit_card,2018-05-02 15:17:41,2018-05-02 15:17:41,0,118,ES,bom jesus do norte


In [58]:
# Export a Supabase
df_ml.to_sql("data_ml", engine, if_exists="replace", index=False)

357